In [28]:
pwd()

"/Users/FredYu/Documents/GitHub/BulkLMM_Analyses/BXDLongevity/output"

In [29]:
using CSV, DelimitedFiles, DataFrames, Missings, XLSX
using LinearAlgebra, Statistics, Optim
using Random, Distributions, LoopVectorization
# using GeneNetworkAPI, Downloads
using BenchmarkTools
using Plots

In [30]:
sampled_traits = readdlm("samples1000.txt", '\t') |> vec |> x -> Int.(x);

In [31]:
function meanAbsDiff(x, y)
    
    return mean(abs.(x .- y))
    
end

meanAbsDiff (generic function with 1 method)

In [50]:
function removeInfValues!(x)
    
    toRemove = findall(isinf.(x));
    
    for inf_val in toRemove
        
        x[inf_val[1], inf_val[2]] = 0.0;
        
    end
    
end

removeInfValues! (generic function with 2 methods)

In [46]:
function removeInfValues!(x, y)
    
    toRemove = findall(isinf.(x));
    
    for inf_val in toRemove
        
        y[inf_val[1], inf_val[2]] = 0.0;
        
    end
    
end

removeInfValues! (generic function with 2 methods)

## Individual LP:

In [32]:
bulklmm_results_ind_null = readdlm("output/bulklmm_L_ind_null.txt", '\t');

In [33]:
bulklmm_results_ind_null_grid = readdlm("output/bulklmm_L_ind_null_grid20.txt", '\t');

In [34]:
bulklmm_results_ind_alt_grid = readdlm("output/bulklmm_L_ind_alt_grid20.txt", '\t');

In [35]:
gemma_results_ind = readdlm("gemma_L_ind.txt", '\t');

In [47]:
removeInfValues!(gemma_results_ind, bulklmm_results_ind_null)

In [48]:
removeInfValues!(gemma_results_ind, bulklmm_results_ind_null_grid)

In [49]:
removeInfValues!(gemma_results_ind, bulklmm_results_ind_alt_grid)

In [51]:
removeInfValues!(gemma_results_ind)

In [52]:
meanAbsDiff(bulklmm_results_ind_null, gemma_results_ind)

0.019844982572864846

In [53]:
meanAbsDiff(bulklmm_results_ind_null_grid, gemma_results_ind)

0.026090870615912806

In [54]:
meanAbsDiff(bulklmm_results_ind_alt_grid, gemma_results_ind)

0.03286788043198145

In [55]:
meanAbsDiff(bulklmm_results_ind_alt_grid, bulklmm_results_ind_null)

0.015995631848133355

In [56]:
meanAbsDiff(bulklmm_results_ind_alt_grid, bulklmm_results_ind_null_grid)

0.010085232015539014

In [59]:
findmax(abs.(bulklmm_results_ind_alt_grid .- gemma_results_ind))

(1.225190904225336, CartesianIndex(201, 52))

In [60]:
bulklmm_results_ind_alt_grid[201, 52]

13.21768068820141

In [61]:
gemma_results_ind[201, 52]

11.992489783976074

In [62]:
thrs = 3.0

3.0

In [67]:
to_reject = findall(gemma_results_ind .> 3.0);
count = 0;

for k in to_reject
    i = k[1];
    j = k[2];
    
    if bulklmm_results_ind_alt_grid[i, j] >= 3.0
        
        count = count+1;
        
    end
end

In [68]:
count

3097

In [69]:
to_reject

3097-element Vector{CartesianIndex{2}}:
 CartesianIndex(2334, 4)
 CartesianIndex(2335, 4)
 CartesianIndex(2336, 4)
 CartesianIndex(2337, 4)
 CartesianIndex(2338, 4)
 CartesianIndex(2339, 4)
 CartesianIndex(2340, 4)
 CartesianIndex(2341, 4)
 CartesianIndex(2342, 4)
 CartesianIndex(2343, 4)
 CartesianIndex(2344, 4)
 CartesianIndex(2345, 4)
 CartesianIndex(2346, 4)
 ⋮
 CartesianIndex(5980, 996)
 CartesianIndex(5981, 996)
 CartesianIndex(5982, 996)
 CartesianIndex(5983, 996)
 CartesianIndex(5984, 996)
 CartesianIndex(5985, 996)
 CartesianIndex(5986, 996)
 CartesianIndex(5987, 996)
 CartesianIndex(5988, 996)
 CartesianIndex(5989, 996)
 CartesianIndex(5990, 996)
 CartesianIndex(5991, 996)

In [ ]:
histogram(bulklmm_results_ind_null .- gemma_results_ind)

## Strain LP:

In [36]:
bulklmm_results_strains_null = readdlm("output/bulklmm_L_strains_null.txt", '\t');

In [37]:
bulklmm_results_strains_null_grid = readdlm("output/bulklmm_L_strains_null_grid20.txt", '\t');

In [38]:
bulklmm_results_strains_alt_grid = readdlm("output/bulklmm_L_strains_alt_grid20.txt", '\t');

In [39]:
gemma_results_strains = readdlm("gemma_L_strains.txt", '\t');

In [57]:
# plot(gemma_results_strains[1:1000, :], bulklmm_results_strains_null[1:1000, :])